__Hedge Calculations__

- 1. Defining lay stake for bet -> lay hedge
- 2. Defining max profitable lay price for bet -> lay hedge 
- 3. Defining back stake for lay -> back hedge
- 4. Defining in profitable back price for lay -> back
- 5. Payout with commission function
- 6. Basic profit simulation in n bets with random prices
- 7. Defining weighted lay stake for bet -> lay long heavy hedge



In [69]:
import numpy as np

In [70]:
back_price = 2.5
back_size = 15
lay_price = 2
commission = 0.05
result = 'win'

def payout(bp, bs, lp, ls, c):
    if ls == '?':
        ls = lay_hedge_stake(bp, bs, lp, c)
    elif bs == '?':
        bs = bet_hedge_stake(lp, ls, bp, c)
    loss_side = - bs + ls * (1 - c) 
    win_side = (bp - 1) * bs * (1 - c) - (lp - 1) * ls
    return win_side 

# 1

In [71]:
def lay_hedge_stake(bp, bs, lp, c):
    """
    Function to identify 
    """
    return (((bp - 1) * bs * (1 - c)) + bs) / (lp - c)

In [72]:
ls = lay_hedge_stake(back_price, back_size, commission, lay_price)
print(ls)

3.8461538461538463


# 2

In [73]:
def lay_hedge_max_lp(bp, c):
    '''
    Given some back price and commission, this returns the maximum lay price for a profitable hedge bet. The lower the lay price, the more profitable the bet.
    '''
    return (bp - 1) * (1 - c) ** 2 + 1

In [74]:
lp_max = lay_hedge_max_lp(2.0, 0.05)
print(str(lp_max) + "\n")

#0 profit bet
print(payout(2.0, 1000, lp_max, '?', 0.05))
#profitable bet, laying at lp_max - 0.01 = 1.890249999
print(payout(2.0, 1000, lp_max - 0.01, '?', 0.05))
print(payout(2.0, 1000, lp_max - 0.03, '?', 0.05))
print(payout(2.0, 1000.0, lp_max - 0.1, '?', 0.05))
print(payout(2.0, 1000.0, lp_max - 0.2, '?', 0.05))

1.9024999999999999

0.0
5.427408412483032
16.460905349794416
57.061340941512185
121.02874432677766


# 3

In [75]:
def bet_hedge_stake(lp, ls, bp, c):
    """
    Hedge functions for when laying first to bet after (short hedge?) a horse
    """
    return ls * (lp - c) / (bp * (1 - c) + c)

# 4

In [76]:
def bet_hedge_min_bp(lp, c):
    '''
    Given some lay price and commission, this returns the minimum back price for a profitable hedge bet. The higher the back price, the more profitable the bet.
    '''
    return (c ** 2 - 2 * c + lp) / (1 - c) ** 2

In [77]:
bp_min = bet_hedge_min_bp(1.91, 0.05)
print(str(bp_min) + "\n")

#0 profit bet
print(payout(bp_min, '?', 1.91, 1000, 0.05))
#profitable bet at bp = 2.018...
print(payout(bp_min + 0.01, '?', 1.91, 1000, 0.05))
print(payout(bp_min + 0.03, '?', 1.91, 1000, 0.05))
print(payout(bp_min + 0.1, '?', 1.91, 1000, 0.05))
print(payout(bp_min + 0.2, '?', 1.91, 1000, 0.05))

2.0083102493074794

1.1368683772161603e-13
4.587284814274767
13.630221374349162
43.9623125240355
84.03577554520984


# 5

In [78]:
def payout_comm(back_price, back_size, lay_price, lay_size, result, commission):
    '''
    Function calculates payout of result based on positions and result.
    '''
    
    if result == 'win':
        back_payout = ((back_price * back_size) - back_size) * (1-commission)
        lay_payout = - ((lay_price * lay_size) - lay_size)
        print(back_payout, lay_payout)
        
    else:
        back_payout = -back_size
        lay_payout = (lay_size) * (1-commission)
        print(back_payout, lay_payout)
        
    return round(round(back_payout, 2) + round(lay_payout, 2), 2)

# 6

In [79]:
"""
taking random prices and determining whether or not to bet with a backing stake of 1,000
"""

from random import uniform
from random import normalvariate

bs = 1000
c = 0.05
total_return = 0

for i in range(1, 6):
    bp = uniform(0.0, 20.0)
    lp = max(normalvariate(bp, 5), 0)
    ls = lay_hedge_stake(bp, bs, lp, c)
    lp_max = lay_hedge_max_lp(bp, c)
    pay = payout(bp, bs, lp, '?', c)
    
    if lp > lp_max:
        print("Bp: %f, lp: %f, no profitable hedge. Bet: £%d and lay £%d would give loss of -£%d. Need lp < %f" % (bp, lp, bs, ls, abs(pay), lp_max))
        print("Total return after %d bets is %d" % (i, total_return))
    elif 0 < lp < lp_max:
        total_return += pay
        print("Bp: %f, lp: %f, profitable hedge exists. Bet: £%d and lay £%d gives profit of £%d." % (bp, lp, bs, ls, pay))
        print("Total return after %d bets is %d" % (i, total_return))

Bp: 8.715921, lp: 8.720259, no profitable hedge. Bet: £1000 and lay £960 would give loss of -£87. Need lp < 7.963618
Total return after 1 bets is 0
Bp: 19.215589, lp: 17.440919, no profitable hedge. Bet: £1000 and lay £1052 would give loss of -£0. Need lp < 17.439569
Total return after 2 bets is 0
Bp: 10.952161, lp: 11.116690, no profitable hedge. Bet: £1000 and lay £944 would give loss of -£102. Need lp < 9.981825
Total return after 3 bets is 0
Bp: 17.073691, lp: 17.649222, no profitable hedge. Bet: £1000 and lay £924 would give loss of -£121. Need lp < 15.506506
Total return after 4 bets is 0
Bp: 10.989399, lp: 16.784962, no profitable hedge. Bet: £1000 and lay £626 would give loss of -£404. Need lp < 10.015432
Total return after 5 bets is 0


# 7

In [82]:
"""
Weighting the backing stake with a factor related to implied odds

tried w = (1 - 1 / (10 * exp(bp)))    #from math import exp #if doing exp()
      w = 1 - (bp - lp) / 100      #this will be a bad reflection of probability
      w = 1 - max((1 / lp - 1 / bp) / 10, 0)
      could change weighting above to rapidly approach 0 where lp is near max hedge price
"""

def weighted_lay_stake(bp, bs, lp, c):
    w = 1 - max((1 / lp - 1 / bp) / 10, 0)
    return ((((bp - 1) * bs * (1 - c)) + bs) / (lp - c) ) * w

for i in [0, 0.03, 0.1, 0.2]:
    ls = weighted_lay_stake(2.0, 1000, lp_max - i, 0.05)
    lp = lay_hedge_max_lp(2.0, 0.05) - i
    bp = 2
    bs = 1000
    c = 0.05
    print((bp - 1) * bs * (1 - c) - (lp - 1) * ls)
    print(str(- bs + ls * (1 - c)) + "\n")


'''
vs in the non-weighted system:
0.0
1.1368683772161603e-13

16.460905349794416
16.46090534979419

57.061340941512185
57.0613409415123

121.02874432677766
121.02874432677777
'''

773.4020434762886
-814.1074141855669

778.7568241333447
-813.5461122368796

791.3779615024521
-812.2231319966722

809.7220335901263
-810.3002589475018



'\nvs in the non-weighted system:\n0.0\n1.1368683772161603e-13\n\n16.460905349794416\n16.46090534979419\n\n57.061340941512185\n57.0613409415123\n\n121.02874432677766\n121.02874432677777\n'